<h2>The RSA crypto scheme, sometimes referred to as the
Rivest–Shamir–Adleman algorithm, is currently the
most widely used asymmetric cryptographic scheme</h2>
<h3> There are many applications for RSA, but in practice it
is most often used for:</h3><br />
1- encryption of small pieces of data, especially for key
transport <br />
2- digital signatures

<br /><h3>Here we are going to implement RSA with public and private keys up to 512 bits within limit of time for setup and decryption less than 3 seconds!<h3>

First let us import our neede libraries

In [1]:
import random
import numpy as np
from random import randrange, randint
import timeit

1- We need to implement Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers

In [2]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

2- Euclid's extended algorithm for finding the multiplicative inverse of two numbers

In [3]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi/e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2- temp1* x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d + phi

3- Tests to see if a number is prime. and to keep our time limit we need to use a trick to make our test faster so we implement Miller-rabin for primality test 

In [4]:
def is_prime(n, k=10):
    if n == 2:
        return True
    if not n & 1:
        return False

    def check(a, s, d, n):
        x = pow(a, d, n)
        if x == 1:
            return True
        for i in xrange(s - 1):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
        return x == n - 1

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    for i in xrange(k):
        a = randrange(2, n - 1)
        if not check(a, s, d, n):
            return False
    return True

4- here we are going to generat prime number according to the size entered bu the user

In [5]:
def generate_big_prime(n, p_in=0):
    found_prime = False
    while not found_prime:
        p = randint(2**(n-1), 2**n)
        if is_prime(p, 100) and p != p_in:
            return p

5- generate public and private key pair 

In [6]:
def generate_keypair(p, q, n):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    n = np.multiply(p , q)
    phi = np.multiply((p-1) , (q-1))
    e = random.randrange(1, phi)
    # Use Euclid's Algorithm to verify that e and phi(n) are comprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    # Return public and private keypair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

6- here we are gonna finish we need Encryption and Decryption methods of our RSA

In [7]:
def encrypt(pk, plaintext):
    #Unpack the key into it's components
    key, n = pk
    cipher = [(pow(ord(char), key , n)) for char in plaintext]
    return cipher

In [8]:
def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char , key , n)) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)

Finally our main method to call previous methods 

In [9]:
    print "RSA Encrypter/ Decrypter"

RSA Encrypter/ Decrypter


In [10]:
    p_size =int(raw_input("Enter size of P in bits : "))
    q_size =int(raw_input("Enter size of q in bits : "))

Enter size of P in bits : 512
Enter size of q in bits : 512


In [11]:
    setup_start = timeit.default_timer()
    p = generate_big_prime(p_size)
    q = generate_big_prime(q_size, p)  

In [12]:
    print('Value of P : ' + str(p))
    print('Value of q : ' + str(q))

Value of P : 7705580158897366208066395286697188178145755156133888488775873841075148792189067613137846039428479422055899613118775687889787385808986281933018004009523183
Value of q : 9571442555175591536738847323825228930649080619823948475304404737143796733131214907982885732599513065663156055428081292277705430780756249004133039144941449


In [13]:
    print "Generating your public/private keypairs now . . ."
    public, private = generate_keypair(p, q, p_size)
    print "Your public key is ", public ," and your private key is ", private

Generating your public/private keypairs now . . .
Your public key is  (51704560030315315082544396437544494702780860358532832490907147085033438317621637416094967194645853819356994299564582976260206046704615738547093831327458147431330691245187089181201853919250108564041599921555586690612975700544434802743717977509663146518034378996710916927646376248426278911852096638961735723325L, 73753517845186947462870839990295867594659684938289269926348686493608242688803772780255972614275324605085455431870293621473161066494414400830017310543189882378960509986565186738023712523605600396396797186444259541601710687150075628444700569962923906433094599305961734252524544696638360388233130971236943112167L)  and your private key is  (8096234485088758383968641181251348833745120223179004426667861030631435360884355986760118892855899894576057194041153112919741400530832615500760277580557889412965250846546119574110581275528523758527604811666400170566420727863419130833187915708317992914310575492872192977164768868850

In [14]:
setup_end = timeit.default_timer()
print("setup time is : " + str((setup_end - setup_start)))

setup time is : 0.820677995682


In [15]:
    message = raw_input("Enter a message to encrypt with public key: ")
    encrypted_msg = encrypt(public, message)
    print "Your encrypted message is: "
    print ''.join(map(lambda x: str(x), encrypted_msg))
    print "Decrypting message with private key  . . ."
    

Enter a message to encrypt with public key: 7705580158897366208066395286697188178145755156133888488775873841075148792189067613137846039428479422055899613118775687889787385808986281933018004009523183
Your encrypted message is: 
...
Decrypting message with private key  . . .


In [16]:
    decrypt_start = timeit.default_timer()
    decrypted_message =  decrypt(private, encrypted_msg)
    decrypt_end = timeit.default_timer()  

In [17]:
    print "Your original massage is: " + str(decrypted_message)
    print("Decryption time is : " + str((decrypt_end - decrypt_start)))

Your original massage is: 7705580158897366208066395286697188178145755156133888488775873841075148792189067613137846039428479422055899613118775687889787385808986281933018004009523183
Decryption time is : 1.26362299919


In [18]:
    print("setup Time + Decryption Time = " + str((decrypt_end - decrypt_start) + (setup_end - setup_start)))

setup Time + Decryption Time = 2.08430099487
